In [2]:
# League of Legends Game State Analyzer with YOLO Label Generation
import cv2
import pytesseract
import os
import time
import numpy as np
from PIL import Image
import pandas as pd

# Configuration
OUTPUT_DIR = "extracted_frames_MR"
MAX_PROCESSING_SECONDS = 600  # 10 minutes
VIDEO_PATH = "videos/MR.mp4"
ANALYSIS_INTERVAL = 1  # Seconds between frames
OUTPUT_IMAGES = True  # Set False to disable image saving

# Region of Interest Configurations (1920x1080 resolution)
UI_CONFIG = {
    "gold": {
        "roi": (1170, 1045, 70, 30),
        "threshold": 180,
        "digits": 4
    },
    "minions": {
        "roi": (1775, 0, 40, 30),
        "threshold": 190,
        "digits": 3
    },
    "skills": {
        "Q": (730, 950, 60, 60),
        "W": (795, 950, 60, 60),
        "E": (860, 950, 60, 60),
        "R": (925, 950, 60, 60)
    }
}

# Cooldown detection parameters
BLUE_HSV_LOWER = np.array([90, 50, 50], dtype=np.uint8)
BLUE_HSV_UPPER = np.array([120, 255, 255], dtype=np.uint8)
CD_THRESHOLD = 0.2

# Skill output directories
SKILL_DIRS = {
    'Q': "q_cd",
    'W': "w_cd",
    'E': "e_cd",
    'R': "r_cd"
}

# Tesseract Setup (Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def create_yolo_labels(skill_img):
    """Generate YOLO format labels for detected digits"""
    # Convert to grayscale and threshold
    gray = cv2.cvtColor(skill_img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours of potential digits
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    labels = []
    
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w*h > 100:  # Filter small noise
            # Convert to YOLO format (normalized coordinates)
            x_center = (x + w/2) / skill_img.shape[1]
            y_center = (y + h/2) / skill_img.shape[0]
            width = w / skill_img.shape[1]
            height = h / skill_img.shape[0]
            
            labels.append(f"0 {x_center:.4f} {y_center:.4f} {width:.4f} {height:.4f}")
    
    return labels

def preprocess_image(roi):
    """Enhanced image optimization pipeline"""
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    _, thresh = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def extract_digits(processed_img, config):
    """OCR execution with validation"""
    custom_config = f'--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789'
    try:
        text = pytesseract.image_to_string(Image.fromarray(processed_img), config=custom_config)
        filtered = ''.join(filter(str.isdigit, text)).lstrip('0')
        return filtered.zfill(config["digits"]) if filtered else '0'*config["digits"]
    except Exception as e:
        print(f"OCR Error: {str(e)}")
        return 'NULL'

def detect_cooldown(skill_roi):
    """Detect cooldown state using original image for OCR"""
    hsv = cv2.cvtColor(skill_roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, BLUE_HSV_LOWER, BLUE_HSV_UPPER)
    coverage = np.sum(mask > 0) / (skill_roi.size / 3)
    
    if coverage > CD_THRESHOLD:
        # Convert to RGB for Tesseract
        skill_roi_rgb = cv2.cvtColor(skill_roi, cv2.COLOR_BGR2RGB)
        
        # OCR on original image
        text = pytesseract.image_to_string(
            skill_roi_rgb,
            config='--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789'
        )
        digits = ''.join(filter(str.isdigit, text))
        return digits or "CD", skill_roi
    
    return "Ready", None

def process_frame(frame, frame_count, video_time_ms):
    """Complete frame processing workflow with video timestamp"""
    # Convert milliseconds to HH:MM:SS.mmm format
    hours, ms_remaining = divmod(video_time_ms, 3600000)
    minutes, ms_remaining = divmod(ms_remaining, 60000)
    seconds, milliseconds = divmod(ms_remaining, 1000)
    timestamp = f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}.{int(milliseconds):03d}"
    
    frame_dir = os.path.join(OUTPUT_DIR, f"frame_{frame_count}")
    marked_frame = frame.copy()
    results = {}
    
    if OUTPUT_IMAGES:
        os.makedirs(frame_dir, exist_ok=True)
        cv2.imwrite(os.path.join(frame_dir, "original.jpg"), frame)

    # Process gold and minions
    for element in ["gold", "minions"]:
        cfg = UI_CONFIG[element]
        x, y, w, h = cfg["roi"]
        roi = frame[y:y+h, x:x+w]
        
        processed = preprocess_image(roi)
        results[element] = extract_digits(processed, cfg)
        
        if OUTPUT_IMAGES:
            cv2.imwrite(os.path.join(frame_dir, f"{element}_processed.jpg"), processed)
            cv2.rectangle(marked_frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Process skills and save training data
    skill_results = {}
    for skill, roi in UI_CONFIG["skills"].items():
        x, y, w, h = roi
        skill_roi = frame[y:y+h, x:x+w]
        
        cd_state, processed_img = detect_cooldown(skill_roi)
        skill_results[skill] = cd_state
        
        # Visualization
        color = (255, 0, 0) if cd_state not in ["Ready"] else (0, 255, 0)
        cv2.rectangle(marked_frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(marked_frame, cd_state, (x+5, y+20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        
        # Save training data for YOLO
        if cd_state not in ["Ready"] and OUTPUT_IMAGES and processed_img is not None:
            skill_dir = os.path.join(OUTPUT_DIR, SKILL_DIRS[skill])
            os.makedirs(skill_dir, exist_ok=True)
            
            # Save image
            img_name = f"{timestamp.replace(':', '-')}_{skill}.jpg"
            img_path = os.path.join(skill_dir, img_name)
            cv2.imwrite(img_path, processed_img)
            
            # Generate and save labels
            labels = create_yolo_labels(processed_img)
            label_path = os.path.join(skill_dir, img_name.replace('.jpg', '.txt'))
            with open(label_path, 'w') as f:
                f.write('\n'.join(labels))

    if OUTPUT_IMAGES:
        cv2.imwrite(os.path.join(frame_dir, "marked.jpg"), marked_frame)

    return {
        "video_timestamp": timestamp,
        **results,
        **skill_results,
        "frame_path": frame_dir
    }

def video_processing():
    """Main processing loop with time constraint"""
    start_time = time.time()
    cap = cv2.VideoCapture(VIDEO_PATH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * ANALYSIS_INTERVAL)
    
    dataset = []
    frame_count = 0
    
    # Create output directories
    for skill_dir in SKILL_DIRS.values():
        os.makedirs(os.path.join(OUTPUT_DIR, skill_dir), exist_ok=True)
    
    while cap.isOpened() and (time.time() - start_time) <= MAX_PROCESSING_SECONDS:
        ret, frame = cap.read()
        if not ret: break
        
        current_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        
        if frame_count % frame_interval == 0:
            frame_data = process_frame(frame, frame_count, current_time_ms)
            log_entry = (f"Video Time: {frame_data['video_timestamp']} | "
                        f"Gold: {frame_data['gold']} | "
                        f"Minions: {frame_data['minions']} | "
                        f"Q: {frame_data['Q']} | "
                        f"W: {frame_data['W']} | "
                        f"E: {frame_data['E']} | "
                        f"R: {frame_data['R']}")
            print(log_entry)
            dataset.append(frame_data)
        
        frame_count += 1
    
    pd.DataFrame(dataset).to_csv(os.path.join(OUTPUT_DIR, "analysis_report.csv"), index=False)
    cap.release()
    print(f"Processing completed. Results saved to {OUTPUT_DIR}")

if __name__ == "__main__":
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    video_processing()

Video Time: 00:00:00.000 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:00.920 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:01.960 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:02.961 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:03.934 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:04.921 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:05.935 | Gold: 0000 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:06.949 | Gold: 0000 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:07.922 | Gold: 0000 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:08.923 | Gold: 0000 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Video Time: 00:00:09.910 | Gol